### CNN

- 합성곱 신경망( CNN : Convolutional Neural Network )

- 1998 : 얀 레쿤 교수 제안
- 이미지 인식 분야에 강력한 성능 발휘
- 음성인식, 자연어 처리에도 사용 → 활용성이 높아지고 있다

- 이미지 인식 대회 ILSVRC (ImageNet Large Scale Recognition Competition)
    - 2010 : NEC-UIUC
    - 2011 : XRCE
    - 2012 : AlexNet(CNN 알고리즘 기반, 딥러닝 시도)
    - 2013 : ZFNet
    - 2014 : GoogLeNet(Inception v2), VGGNet 더 유명(간결함, 편의성)
    - 2015 : ResNet → 사람의 정확도(오차율 5%)를 돌파
    - 2016 : GoogLeNet-v4
    - 2017 : SENet → 2.3%

<img src = './data/7.ILSVRC_랭킹.png' width = '400'>

<img src = './data/5.CNN_도식.png'>

- 기본 구성
    - [ 입력층 ] →
    - [ 중간층 : [합성곱층][풀링층]...[합성곱층][풀링층][전결합층] ] →
    - [ 출력층 ]

- 중간층
    - 합성곱 은닉 계층
        - 합성곱층(Convolution Layer)
        - 풀링층(Pooling Layer)
    - 전결합층(완전 연결 은닉 계층)
<img src = './data/3.CNN_중간층.jpeg'>    

- 합성곱층
    - 이미지의 특징을 추출
    - 입력 x의 이미지 일부분을 조금씩 잘라가면서, 가중치 필터(W)를 적용(평활화, 윤곽선 검출)
        - 평활화 : 명암의 분포를 균일하게 처리
        - 윤곽선 검출 : 이미지 내부 대상들의 윤곽만 추출
    - 움직이는 크기 한 픽셀에서 n칸으로 이동 → 스트라이드(stride)
    - 가중치 필터 W = kernel(커널), 편향 b(바이어스, bias) 필요
    - 입력층 28x28이라면 784개에 대한 가중치 필요 → 연산량도 많고, 시간도 많이 걸리므로 비효율적 → **컨볼루션 계층 도입** → 3x3으로 줄이면, 가중치는 9개만 있으면 되므로 계산량과 학습량이 줄어들어 효율적
    - 커널이 1개면 비효율적 → **하이퍼 파라미터 조정을 통해서 커널의 수, 크기 등을 조절하여 계층 처리**

<img src = './data/dp1.png'>

- 풀링층
    - 합성곱층으로 얻는 특징 맵 C를 축소하는 층
    - 특징을 유지한 상태로 축소
    - 직선 인식 → 직선이 미세하게 흐트러지더라도 직선으로 인지한다
    - 축소방법 : 최대 풀링, 최소 풀링, 평균 풀링
    <img src = './data/dp2.png'>

- 전결합층
    - 각 층의 유닛 통합
    - 2차원 특징 맵들을 1차원으로 전개
    - 활성화 함수(relu, sigmoid)가 사용되고, 특성을 더욱 강조

In [1]:
import tensorflow as tf

C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
# 데이터 수집
mnist = tf.keras.datasets.mnist.load_data( path='mnist.npz' )

In [ ]:
len(mnist), len(mnist[0])

In [ ]:
# 훈련용
len(mnist[0][0]), len(mnist[0][1])

In [ ]:
# 테스트용
len(mnist[1][0]), len(mnist[1][1])

In [ ]:
# 레이블 확인
mnist[0][1][:2]  # 벡터화 진행

In [ ]:
# 텐서플로우의 샘플에서 획득
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
mnist = input_data.read_data_sets('./data/mnist/', one_hot=True)

In [ ]:
# 훈련용
mnist.train.images.shape, mnist.train.labels.shape

In [ ]:
mnist.train.labels[:2]

In [ ]:
import numpy as np

In [ ]:
# 특정 조건을 만족하는 인덱스
print(np.where(mnist.train.labels[0])[0][0])

In [ ]:
# 784의 제곱근
pixel_size = int(np.sqrt( mnist.train.images.shape[1] ))
pixel_size

In [ ]:
# 이미지 1개에 대한 레이블 크기
pixels = mnist.train.images.shape[1]
nums = mnist.train.labels.shape[1]
pixels, nums

### 텐서플로우의 CNN 작업

In [ ]:
# 입력 채널 → 손글씨 이미지 데이터 → n개(None)
# 이미지 1개당 특성(픽셀) pixels개
x = tf.placeholder( tf.float32, shape = [None, pixels], name = 'x' )

In [ ]:
# 출력 → 손글씨가 0 - 9로 레이블값 : 데이터 n개(None), 출력의 종류, 분류개수(nums)
y_ = tf.placeholder(tf.float32, shape = [None, nums], name = 'y_')
y_

In [ ]:
# (가중치, 필터, 커널)을 초기화하는 함수 구현
# name을 붙인 이유는 합성곱층마다 사용이 될 것이므로, 이를 구분하기 위함
# → 텐서보드에서 그래프를 구분하기 위함
def weight_variable( name, shape ) :
    
    # 절단 정규 분포 함수
    # 평균값을 기준으로 표준편차보다 크거나 작은 데이터는 제외하는 난수 생성
    # stddev : 표준편차
    W_init = tf.truncated_normal(shape, stddev=0.1)
    # 가중치 변수 생성
    W = tf.Variable( W_init, name = 'W_' + name )
    return W

In [ ]:
# 바이어스(편향)
# 필터를 통과한 값을 전체적으로 올리거나, 내리거나
def bias_variable(name, size) :
    # 0.1 임시값 - 파라미터에 따라 다르게 나온다
    b_init = tf.constant( 0.1, shape = [size] )
    b = tf.Variable( b_init, name = 'b_'+ name )
    return b

In [ ]:
# 합성곱 계층 생성
def conv2d(x, W) :
    # 스트라이드 : 커널(필터, 가중치) 얼마 단위로 이동시킬 것인가
    return tf.nn.conv2d( x, W, strides = [1, 1, 1, 1], padding = 'SAME' )

In [ ]:
# 합성곱층 1 생성
# 텐서보드 상에서 그래프에 관련된 범위(scope)를 부여하여 관계를 쉽게 확인
c_name1 = 'conv1'
with tf.name_scope(c_name1) as scope :
    # 5, 32는 설정값
    # 가중치 or 필터 or 커널 생성
    # W_conv1 = weight_variable( c_name1, [ filter_height, filter_width, in_channels, out_channels ] )
    W_conv1 = weight_variable( c_name1, [5, 5, 1, 32] )
    # 편향/바이어스 생성
    b_conv1 = bias_variable( c_name1, 32 )
    # 입력 데이터에 대한 행렬 준비
    # x_img = tf.reshape( x, [batch, in_height, in_width, in_channels] )
    x_img = tf.reshape(x, [-1, pixel_size, pixel_size, 1])
    # 컨볼루젼 생성
    # 활성화 함수를 사용 마감처리
    h_conv1 = tf.nn.relu( conv2d( x_img, W_conv1 ) + b_conv1 )

In [ ]:
h_conv1.get_shape

In [ ]:
# 풀링층 생성 → 크기를 대폭 줄여서, 데이터량을 줄이고 특징은 유지
# 최대 풀링, 최소 풀링 등 존재
# x의 shape : (?, 28, 28, 32)
def max_pool(x) :
    # 2, 2는 설정값
    return tf.nn.max_pool( x, ksize = [1, 2, 2, 1],
                          strides = [1, 2, 2, 1], padding = 'SAME' )

tf.nn.max_pool(
    value,
    ksize,
    strides,
    padding,
)

In [ ]:
with tf.name_scope('pool1') as scope :
    h_pool1 = max_pool( h_conv1 )

In [ ]:
# (?, 14, 14, 32) 
# 이전층 대비 가로, 세로 길이가 반으로 줄었다
h_pool1.get_shape

In [ ]:
# 합성곱층2 생성 → 출력 : 채널이 64로 늘어난다

c_name2 = 'conv2'
with tf.name_scope(c_name2) as scope :
    W_conv2 = weight_variable( c_name2, [5, 5, 32, 64] )
    b_conv2 = bias_variable( c_name2, 64 )
    h_conv2 = tf.nn.relu( conv2d( h_pool1, W_conv2 ) + b_conv2 )

In [ ]:
# 풀링층2 생성 → 가로, 세로길이가 7로, 즉 반으로 줄어든다
with tf.name_scope('pool2') as scope :
    h_pool2 = max_pool( h_conv2 )

In [ ]:
# (?, 7, 7, 64)
h_pool2.get_shape

In [ ]:
# 전결합층
# 현재 이미지의 크기는 합성곱층을 2번 통과했으므로 
# → 28/2/2 = 7

In [ ]:
fc_name = 'fc'
with tf.name_scope('fully_connected_layer') as scope :
    
    n = 7 * 7 * 64
    # 가중치
    # 출력 채널을 1024개로 설정
    W_fc = weight_variable( fc_name, [n, 1024] )
    # 편향
    b_fc = bias_variable( fc_name, 1024 )
    # [-1, n] * [n, 1024] + [1024] = [-1, 1024]
    h_pool2_fc = tf.reshape( h_pool2, [-1, n] )
    # 활성화 함수 통과
    h_fc = tf.nn.relu(tf.matmul( h_pool2_fc , W_fc ) + b_fc)

In [ ]:
# (?, 1024)
h_fc.get_shape

In [ ]:
# 과잉 적합 막기(옵션)
with tf.name_scope('dropout') as scope :
    keep_prob = tf.placeholder( tf.float32 )
    h_fc_drop = tf.nn.dropout( h_fc, rate = 1-keep_prob )

In [ ]:
# (?, 1024)
h_fc_drop.get_shape

In [ ]:
# 출력층 구성 - 활성화 함수로 softmax 사용
ro_name = 'read_out'
with tf.name_scope(ro_name) as scope :
    W_ro = weight_variable( ro_name, [1024, 10]) 
    b_ro = bias_variable( ro_name, 10 )
    y_conv = tf.nn.softmax( tf.matmul( h_fc_drop, W_ro ) + b_ro )

In [ ]:
# (?, 10)
y_conv.get_shape

In [ ]:
# 모델 학습
# 손실값 정의
with tf.name_scope('loss') as scope :
    # 크로스 엔트로피
    # 비용(cost)/손실(loss) : 원하는 결과에 얼마나 떨어져 있는가, 이 격차를 줄인다
    cross_entropy = -tf.reduce_sum(y_ * tf.log(y_conv))

In [ ]:
# 모델 훈련
with tf.name_scope('training') as scope :
    # 확률적 경사 하강법
    # 무작위로 초기화한 매개변수를 손실함수가 작아지도록 지속적으로 반복하여 변경
    optimizer = tf.train.AdamOptimizer( 1e-4 )
    train_step = optimizer.minimize( cross_entropy )

In [ ]:
# 모델 평가
with tf.name_scope('predict') as scope :
    predict_step = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    accuracy_step = tf.reduce_mean(tf.cast(predict_step, tf.float32))

In [ ]:
# 데이터를 훈련 및 예측 평가 시 사용할 수 있는 구조로 변형
def set_feed( images, labels, prob ) :
    return { x : images, y_: labels, keep_prob : prob }

In [ ]:
# 세션 시작 → 연산 시작
with tf.Session() as sess :
    # 텐서플로우 변수 초기화
    sess.run(tf.global_variables_initializer())
    
    # 테스트 전용 피드 데이터
    test_fd = set_feed( mnist.test.images, mnist.test.labels, 1 )
    
    # 학습
    for step in range( 3000 ) : # 10000 ) :
        # 데이터를 50개씩 사용하겠다
        # 훈련용 데이터에서 50개 획득
        batch = mnist.train.next_batch(50)
        # 0.5는 설정값
        fd = set_feed( batch[0], batch[1], 0.5 )
        # 훈련
        _, loss = sess.run( [ train_step , cross_entropy ], feed_dict = fd  )
        # 100번째마다 출력
        if step % 100 == 0 :
            acc = sess.run( accuracy_step, feed_dict = test_fd )
            print('step = %s loss = %s acc = %s' %(step, loss, acc))
    # 최종 결과 출력
    acc = sess.run( accuracy_step, feed_dict = test_fd )
    # print('step = %s loss = %s acc = %s' %(step, loss, acc))
    print('정확도 =', acc)
    
    # 텐서보드 기록
    tf.summary.FileWriter('./log_tf_cnn', graph = sess.graph)
    
    # 종료 후
    # $ tensorboard --logdir=log_tf_cnn